#### 앙상블 -> voting 방식
- 여러개의 모델 또는 동일 모델과 샘플링 데이터셋으로 병렬학습 진행하는 방식
- voting 방식/기법
    - 구성: 동일 데이터셋 + 다른 알고리즘 모델
    - 결과도출: Hard(직점), soft(간접)

- 유방암 판별 모델 구현 => 피쳐: 30개, 타겟: 2개(2진분류)


In [49]:
# [1] 데이터 준비
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np

# 데이터 로딩
dataSet= load_breast_cancer(as_frame=True)
print(dataSet.keys())
dataSet.target_names

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


array(['malignant', 'benign'], dtype='<U9')

In [50]:
# 데이터 확인
print(f'target_names: {dataSet.target_names}')
print(F'features names: {dataSet.feature_names}')
print(F'DESCR: {dataSet.DESCR}')

target_names: ['malignant' 'benign']
features names: ['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
DESCR: .. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (stand

In [51]:
# 피쳐와 타겟 저장
featureDF=dataSet['data']
targetSR=dataSet['target']
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(featureDF, targetSR, stratify=targetSR,
                                                   random_state=10)

In [52]:
# KN, LR, DT 알고리즘 사용
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

k_model=KNeighborsClassifier()
dt_model=DecisionTreeClassifier()
lr_model=LogisticRegression(solver='liblinear')


In [60]:
# 보팅 인스턴스 생성
v_model=VotingClassifier(estimators=[('k_model', k_model), ('dt_model', dt_model), ('lr_model', lr_model)],
                         voting='hard')
vs_model=VotingClassifier(estimators=[('k_model', k_model), ('dt_model', dt_model), ('lr_model', lr_model)],
                         voting='soft')

In [61]:
v_model.fit(X_train.values, Y_train.values)

vs_model.fit(X_train.values, Y_train.values)


VotingClassifier(estimators=[('k_model', KNeighborsClassifier()),
                             ('dt_model', DecisionTreeClassifier()),
                             ('lr_model',
                              LogisticRegression(solver='liblinear'))],
                 voting='soft')

In [55]:
# 모델 파라미터 확인
print(f'v_model.classes_: {v_model.classes_}')
print(f'v_model.estimators: {v_model.estimators}')
print(f'v_model.named_estimators: {v_model.named_estimators}')
# print(f'v_model.n_features_in_: {v_model.n_features_in_}')
# print(f'v_model.feature_names_in_: {v_model.feature_names_in_}')

v_model.classes_: [0 1]
v_model.estimators: [('k_model', KNeighborsClassifier()), ('dt_model', DecisionTreeClassifier()), ('lr_model', LogisticRegression(solver='liblinear'))]
v_model.named_estimators: {'k_model': KNeighborsClassifier(), 'dt_model': DecisionTreeClassifier(), 'lr_model': LogisticRegression(solver='liblinear')}


In [62]:
# [4] 성능 확인 => trainDS와 testDS
test_score=v_model.score(X_test.values, Y_test.values)
train_score=v_model.score(X_train.values, Y_train.values)
print(train_score, test_score)

test_score=vs_model.score(X_test.values, Y_test.values)
train_score=vs_model.score(X_train.values, Y_train.values)
print(train_score, test_score)

0.9741784037558685 0.951048951048951
0.9882629107981221 0.951048951048951
